In [27]:
from roboflow import Roboflow
from ultralytics import YOLO
import opendatasets
import os
import shutil

In [43]:
rf = Roboflow(api_key="API_KEY")
project = rf.workspace("bike-helmets").project("bike-helmet-detection-2vdjo")
dataset = project.version(1).download("yolov8")

f = open("Bike-Helmet-Detection-1/data.yaml", "w")
f.write("names:\n")
f.write("- With_Helmet\n")
f.write("- Without_Helmet\n")
f.write("nc: 2\n")
f.write("test: test/images\n")
f.write("train: train/images\n")
f.write("val: valid/images\n")
f.close()

In [49]:
!cd ./Bike-Helmet-Detection-1 && yolo task=detect mode=train model=yolov8m.pt data=data.yaml epochs=50 imgsz=200 plots=True

100%|██████████████████████████████████████| 49.7M/49.7M [00:04<00:00, 10.7MB/s]
New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, re

In [26]:
project = 'https://www.kaggle.com/savanagrawal/detect-person-on-motorbike-or-scooter'
opendatasets.download(project)

f = open("detect-person-on-motorbike-or-scooter/data.yaml", "w")
f.write("names:\n")
f.write("- Person_Bike\n")
f.write("nc: 1\n")
f.write("test: test/images\n")
f.write("train: train/images\n")
f.write("val: valid/images\n")
f.close()

folder_path = "./detect-person-on-motorbike-or-scooter/dataset/obj"
train_image_folder = "./detect-person-on-motorbike-or-scooter/train/images"
train_text_folder = "./detect-person-on-motorbike-or-scooter/train/labels"
valid_image_folder = "./detect-person-on-motorbike-or-scooter/valid/images"
valid_text_folder = "./detect-person-on-motorbike-or-scooter/valid/labels"
test_image_folder = "./detect-person-on-motorbike-or-scooter/test/images"
test_text_folder = "./detect-person-on-motorbike-or-scooter/test/labels"
train = len([entry for entry in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, entry))])/2
test_and_val = int(train/8)
train = train - test_and_val*2

os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(train_text_folder, exist_ok=True)
os.makedirs(valid_image_folder, exist_ok=True)
os.makedirs(valid_text_folder, exist_ok=True)
os.makedirs(test_image_folder, exist_ok=True)
os.makedirs(test_text_folder, exist_ok=True)

i=0
for filename in os.listdir(folder_path):
    i+=1
    if i < train:
        image_folder = train_image_folder
        text_folder = train_text_folder
    elif i < train + test_and_val:
        image_folder = valid_image_folder
        text_folder = valid_text_folder
    else:
        image_folder = test_image_folder
        text_folder = test_text_folder

    file_path = os.path.join(folder_path, filename)
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        new_filename = str(len(os.listdir(image_folder)) + 1) + os.path.splitext(filename)[1]
        new_image_path = os.path.join(image_folder, new_filename)
        shutil.move(file_path, new_image_path)

    elif filename.endswith('.txt'):
        new_filename = str(len(os.listdir(text_folder)) + 1) + os.path.splitext(filename)[1]
        new_text_path = os.path.join(text_folder, new_filename)
        shutil.move(file_path, new_text_path)

In [48]:
!cd ./detect-person-on-motorbike-or-scooter && yolo task=detect mode=train model=yolov8m.pt data=data.yaml epochs=50 imgsz=200 plots=True

100%|██████████████████████████████████████| 49.7M/49.7M [00:04<00:00, 11.4MB/s]
New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./data.yaml, epochs=50, patience=50, batch=16, imgsz=200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, 

In [46]:
rf = Roboflow(api_key="API_KEY")
project = rf.workspace("augmented-startups").project("vehicle-registration-plates-trudk")
dataset = project.version(1).download("yolov8")

f = open("Vehicle-Registration-Plates-1/data.yaml", "w")
f.write("names:\n")
f.write("- License_Plate\n")
f.write("nc: 1\n")
f.write("test: test/images\n")
f.write("train: train/images\n")
f.write("val: valid/images\n")
f.close()

In [47]:
!cd ./Vehicle-Registration-Plates-1 && yolo task=detect mode=train model=yolov8m.pt data=data.yaml epochs=25 imgsz=200

New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.222 🚀 Python-3.9.18 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=Vehicle-Registration-Plates-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False,

In [ ]:
helmet_model = YOLO("./Bike-Helmet-Detection-1/runs/detect/train/weights/best.pt")
person_bike_model = YOLO("./detect-person-on-motorbike-or-scooter/runs/detect/train/weights/best.pt")
number_plate_model = YOLO("./Vehicle-Registration-Plates-1/runs/detect/train/weights/best.pt")
helmet_model.export(format='saved_model')
person_bike_model.export(format='saved_model')
number_plate_model.export(format='saved_model')